# Visualization and Filtering with Lonboard
This notebook demonstrates how to load the intermediate filtered data and visualize it using Lonboard's ArcLayer.

## Step 1: Install Required Packages
Ensure you have the required packages installed.

In [ ]:
!pip install geopandas polars lonboard pyarrow numpy

## Step 2: Import Libraries and Initialize Sidecar

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pyarrow as pa
import polars as pl
from IPython.display import display, Markdown
from sidecar import Sidecar
import lonboard as lb
from lonboard.experimental import ArcLayer
from lonboard.layer_extension import BrushingExtension

## Step 3: Load Filtered Data

In [7]:
filtered_data_path = '/home/nls/data/mitma/geo/viajes_distritos100k_od.parquet'
filtered_df = pl.read_parquet(filtered_data_path).to_pandas()
display(Markdown("### Filtered data loaded successfully."))
print(filtered_df.head(2))

### Filtered data loaded successfully.

      fecha  periodo origen destino distancia actividad_origen  \
0  20230101        0  01002   01002     0.5-2             casa   
1  20230101        0  01002   01002     0.5-2             casa   

  actividad_destino estudio_origen_posible estudio_destino_posible  \
0         frecuente                     no                      no   
1   trabajo_estudio                     no                      no   

   residencia  ...     ID origen_lat origen_lng  \
0           1  ...  01002  43.025486  -2.971678   
1           1  ...  01002  43.025486  -2.971678   

                                        geometry_wkt  destino_right ID_right  \
0  MULTIPOLYGON (((-2.975247238420685 42.93816416...          01002    01002   
1  MULTIPOLYGON (((-2.975247238420685 42.93816416...          01002    01002   

  destino_lat  destino_lng                                 geometry_wkt_right  \
0   43.025486    -2.971678  MULTIPOLYGON (((-2.975247238420685 42.93816416...   
1   43.025486    -2.971678  MULTI

## Step 4: Filter Out Rows Where 'origen' Equals 'destino'

In [8]:
filtered_df = filtered_df.filter(filtered_df['origen'] != filtered_df['destino'])
display(Markdown("### Filtered out rows where 'origen' equals 'destino'."))

### Filtered out rows where 'origen' equals 'destino'.

## Step 5: Create ArcLayer for Visualization
Create the ArcLayer using the filtered data and visualize it using Lonboard.

In [4]:
value = np.ones(len(filtered_df))  # Use a constant value for simplicity
get_source_position = np.array(filtered_df[['origen_lng', 'origen_lat']].values.tolist())
get_target_position = np.array(filtered_df[['destino_lng', 'destino_lat']].values.tolist())
table = pa.table({'value': value})



In [9]:

brushing_extension = BrushingExtension()
arc_layer = ArcLayer(
    table=table,
    get_source_position=get_source_position,
    get_target_position=get_target_position,
    get_source_color=[0, 128, 200],
    get_target_color=[200, 0, 80],
    get_width=1,
    opacity=0.4,
    pickable=False,
    extensions=[brushing_extension],
    brushing_radius=500,
)


In [10]:

# Initialize the sidecar for the map
sc = Sidecar(title='Lonboard Map')


In [11]:

# Configure Lonboard view
lonboard_map = lb.Map(
    layers=[arc_layer],picking_radius=10

    # initial_view_state={
    #     'latitude': filtered_df['origen_lat'].mean(),
    #     'longitude': filtered_df['origen_lng'].mean(),
    #     'zoom': 10,
    #     'bearing': 0,
    #     'pitch': 45,
    # },
    # tooltip={
    #     'html': 'Trip from {origen_lat}, {origen_lng} to {destino_lat}, {destino_lng}',
    #     'style': {'color': 'white'}
    # }
)

# Display the map in the sidecar
with sc:
    display(lonboard_map)